In [2]:
import os
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)

C:\Users\DELL\anaconda3\envs\tensorflow\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# 读取数据
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load("train_data .csv")

In [10]:
# 获取公司类型有哪些
firms = df.select(df.employer_type).distinct()
firms.show()

+--------------+
| employer_type|
+--------------+
|幼教与中小学校|
|      上市企业|
|      政府机构|
|    世界五百强|
|  高等教育机构|
|      普通企业|
+--------------+



In [11]:
# 将其转化为Pandas，由于一共就几个，这一段可以用Pandas来做
firms = firms.toPandas()
firms.iloc[0, 0]

'幼教与中小学校'

In [12]:
# 将Pandas存入list，每一项为string
firms_list = []
for i in range(len(firms)):
    firms_list.append(firms.iloc[i, 0])
firms_list

['幼教与中小学校', '上市企业', '政府机构', '世界五百强', '高等教育机构', '普通企业']

In [27]:
# 统计数量
num = []
for item in firms_list:
    num_temp = df.filter(df.employer_type == item).count()
    num.append(num_temp)
num

[29995, 30038, 77446, 16112, 10106, 136303]

In [28]:
# 计算结果，保留5位小数，打印结果
total_num = 0
for i in range(len(num)):
    total_num += num[i]
for i in range(len(num)):
    num[i] /= total_num
    num[i] = round(num[i], 5)
for i in range(len(num)):
    print(firms_list[i] + '：  ' + str(num[i]))

幼教与中小学校：  0.09998
上市企业：  0.10013
政府机构：  0.25815
世界五百强：  0.05371
高等教育机构：  0.03369
普通企业：  0.45434


In [31]:
# 转为Pandas的DataFrame，写入文件
import pandas as pd
results = pd.DataFrame({'公司类型': firms_list, '类型占比': num})
results.to_csv('公司类型占比.csv', index = None)

In [36]:
# 增加利息金额列，展示，存入文件
interest = df.withColumn('total_money', df.year_of_loan * df.monthly_payment * 12 - df.total_loan).select('user_id','total_money')
interest.show()
interest = interest.toPandas()
interest.to_csv('利息金额.csv', index = None)

+-------+------------------+
|user_id|       total_money|
+-------+------------------+
|      0|            3846.0|
|      1|1840.6000000000004|
|      2|10465.600000000002|
|      3|1758.5200000000004|
|      4| 1056.880000000001|
|      5| 7234.639999999999|
|      6| 757.9200000000001|
|      7| 4186.959999999999|
|      8|2030.7600000000002|
|      9|378.72000000000116|
|     10| 4066.760000000002|
|     11|1873.5599999999977|
|     12| 5692.279999999999|
|     13|1258.6800000000003|
|     14|6833.5999999999985|
|     15| 9248.200000000004|
|     16| 6197.119999999995|
|     17|1312.4400000000005|
|     18| 5125.200000000001|
|     19|1215.8400000000001|
+-------+------------------+
only showing top 20 rows



In [37]:
# 观察工作年限
df.select(df.work_year).distinct().show()

+---------+
|work_year|
+---------+
|  5 years|
|  9 years|
|     null|
|   1 year|
|  2 years|
|  7 years|
|  8 years|
|  4 years|
|  6 years|
|  3 years|
|10+ years|
| < 1 year|
+---------+



In [39]:
# 将符合的工作年限整入一个list
work_years = df.select(df.work_year).distinct().toPandas()
work_list = []
num_list = [1, 5, 6, 8, 10]
for i in num_list:
    work_list.append(work_years.iloc[i, 0])
work_list

['9 years', '7 years', '8 years', '6 years', '10+ years']

In [69]:
# 使用filter提取，并选取所需的三列
df_new = df.filter(df.work_year.isin(work_list) == True).select('user_id', 'censor_status', 'work_year')
df_new.show()

+-------+-------------+---------+
|user_id|censor_status|work_year|
+-------+-------------+---------+
|      1|            2|10+ years|
|      2|            1|10+ years|
|      5|            2|10+ years|
|      6|            0|  8 years|
|      7|            2|10+ years|
|      9|            0|10+ years|
|     10|            2|10+ years|
|     15|            1|  7 years|
|     16|            2|10+ years|
|     17|            0|10+ years|
|     18|            1|10+ years|
|     20|            1|  7 years|
|     21|            2|10+ years|
|     25|            2|10+ years|
|     26|            0|10+ years|
|     30|            0|10+ years|
|     31|            0|  6 years|
|     33|            1|10+ years|
|     38|            0|10+ years|
|     39|            1|10+ years|
+-------+-------------+---------+
only showing top 20 rows



In [72]:
# 使用withColumn构造新列，将work_year的字符串中只保留数字部分
df_new = df_new.withColumn('work_year', df.work_year[0:2])
df_new.show()

+-------+-------------+---------+
|user_id|censor_status|work_year|
+-------+-------------+---------+
|      1|            2|       10|
|      2|            1|       10|
|      5|            2|       10|
|      6|            0|       8 |
|      7|            2|       10|
|      9|            0|       10|
|     10|            2|       10|
|     15|            1|       7 |
|     16|            2|       10|
|     17|            0|       10|
|     18|            1|       10|
|     20|            1|       7 |
|     21|            2|       10|
|     25|            2|       10|
|     26|            0|       10|
|     30|            0|       10|
|     31|            0|       6 |
|     33|            1|       10|
|     38|            0|       10|
|     39|            1|       10|
+-------+-------------+---------+
only showing top 20 rows



In [73]:
df_new = df_new.toPandas()
df_new.to_csv('censor_status.csv', index = None)